# 1. Scraping the complete Bitcoin Data from 2009

BItcoin Data Source: https://bitcoin.zorinaq.com/price/

In [7]:
# Load the packages
from parse import compile
import pandas as pd
import csv

In [8]:
# Read and parse the data.
pattern = compile('[new Date("{}"), {:f}],')

data = []
with open('data/data.txt') as f:
    for line in f.readlines():
        result = pattern.parse(line.strip())
        data.append(
            list(result) # appending parsed data
        )

In [9]:
# Write the data out in a file
with open('data/bitcoin.csv', 'w', newline='') as f:
    csvwriter = csv.writer(f)
    
    # headers
    csvwriter.writerow(['date', 'close'])
    
    # write out by each line
    for entry in data:
        csvwriter.writerow(entry)

In [10]:
# Check the result
df_bitcoin = pd.read_csv('data/bitcoin.csv') 
df_bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4216 non-null   object 
 1   close   4216 non-null   float64
dtypes: float64(1), object(1)
memory usage: 66.0+ KB


In [11]:
# Check missings
df_bitcoin.isna().sum()

date     0
close    0
dtype: int64

# 2. Exogenous variables

In [12]:
# Read the data in
df_gold = pd.read_csv('data/gold.csv') # Gold
df_oil = pd.read_csv('data/crude_oil.csv') # Crude Oil
df_spx = pd.read_csv('data/spx.csv') # S&P 500
df_vfh = pd.read_csv('data/vfh.csv') # Vanguard Financials Index Fund ETF
df_vgt = pd.read_csv('data/vgt.csv') # Vanguard Information Technology Index Fund ETF
df_nvda = pd.read_csv('data/nvda.csv') # NVIDIA

In [13]:
# Column names: remove white spaces, convert to lower case, add _ instead of space
df_gold.columns = df_gold.columns.str.strip().str.lower().str.replace(' ','_')
df_oil.columns = df_oil.columns.str.strip().str.lower().str.replace(' ','_')
df_spx.columns = df_spx.columns.str.strip().str.lower().str.replace(' ','_')
df_vfh.columns = df_vfh.columns.str.strip().str.lower().str.replace(' ','_')
df_vgt.columns = df_vgt.columns.str.strip().str.lower().str.replace(' ','_')
df_nvda.columns = df_nvda.columns.str.strip().str.lower().str.replace(' ','_')

In [14]:
# Change date to date_time
df_gold['date'] = pd.to_datetime(df_gold['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_spx['date'] = pd.to_datetime(df_spx['date'])
df_vfh['date'] = pd.to_datetime(df_vfh['date'])
df_vgt['date'] = pd.to_datetime(df_vgt['date'])
df_nvda['date'] = pd.to_datetime(df_nvda['date'])

In [15]:
# Change date to Index
df_gold = df_gold.set_index('date')
df_oil = df_oil.set_index('date')
df_spx = df_spx.set_index('date')
df_vfh = df_vfh.set_index('date')
df_vgt = df_vgt.set_index('date')
df_nvda = df_nvda.set_index('date')

In [16]:
# Subset the data frames
df_gold = df_gold.loc['2009-11-01':'2021-03-31']
df_oil = df_oil.loc['2009-11-01':'2021-03-31']
df_spx = df_spx.loc['2009-11-01':'2021-03-31']
df_vfh = df_vfh.loc['2009-11-01':'2021-03-31']
df_vgt = df_vgt.loc['2009-11-01':'2021-03-31']
df_nvda = df_nvda.loc['2009-11-01':'2021-03-31']

In [17]:
# Create a big df
df_exo = pd.DataFrame(df_gold, columns={'close'})
df_exo.rename(columns = {'close' : 'gold'}, inplace = True)

df_exo['oil'] = df_oil['close']
df_exo['spx'] = df_spx['close']
df_exo['vfh'] = df_vfh['close']
df_exo['vgt'] = df_vgt['close']
df_exo['nvda'] = df_nvda['close']

In [18]:
# Show data
df_exo.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2873 entries, 2009-11-02 to 2021-03-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gold    2846 non-null   float64
 1   oil     2847 non-null   float64
 2   spx     2870 non-null   float64
 3   vfh     2870 non-null   float64
 4   vgt     2870 non-null   float64
 5   nvda    2870 non-null   float64
dtypes: float64(6)
memory usage: 157.1 KB


In [19]:
# Deal with missings
df_exo.isna().sum()
df_exo = df_exo.dropna()
df_exo.isna().sum()

gold    0
oil     0
spx     0
vfh     0
vgt     0
nvda    0
dtype: int64

In [20]:
# Write out to a file
df_exo.to_csv('data/exogenous.csv')